## 1) Adding Coord (UTM31 ETRS1989, ETRS1989) and Export to SHP and Excel DWG file


In [4]:
# Import arcpy module
import arcpy
import numpy
import os
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import widgets
from IPython.display import clear_output

#### A) AddCoord [UTM31 ETRS1989, ETRS1989]
Specifying the paths for the input feature class and the directory for Excel output.

In [1]:
# Set the input feature class
in_fc = r"\\WM20ocqu46ph01\WF_Projects\NL_HKW\2_FINAL\INNOVATION\INNOVATION.gdb\HKW_VII_Subsea_Battery_pt_UTM31N_v2"

# Define the directory for Excel output
directory_excel = r"\\WM20ocqu46ph01\WF_Projects\NL_HKW\4_OUTPUT\_Innovation\HKW_VII_Subsea_Battery_pt_UTM31N_v2"


#### B) To delete attribute fields from a feature class 

In [3]:
# Delete the fields and Set the field names to delete
del_fields = ["X_ETRS_UTM31", "Y_ETRS_UTM31","X_ETRS", "Y_ETRS", 'POINT_X', 'POINT_Y']

arcpy.DeleteField_management(in_fc, del_fields)

# Get remaining fields after deletion
fields = [field.name for field in arcpy.ListFields(in_fc)]
print('All Fields:',fields)

arcpy.DeleteField_management(in_fc, del_fields)

# Create a search cursor to fetch data from the feature class
fields = [field.name for field in arcpy.ListFields(in_fc)]
print('Fields:',fields)


Fields: ['OBJECTID', 'Shape']
Fields: ['OBJECTID', 'Shape']


In [4]:
# ETRS 1989 UTM Zone 31N - 25831
# Set the output coordinate system
out_sr = arcpy.SpatialReference(25831) 

# Add geometry attributes
geom_props = "POINT_X_Y_Z_M"
arcpy.AddGeometryAttributes_management(in_fc, geom_props , "", "", out_sr)



# Rename the fields POINT_X and POINT_Y to X_COORD and Y_COORD
arcpy.AlterField_management(in_fc, "POINT_X", "X_ETRS_UTM31", "Easting [ETRS 1989 UTM31] ")
arcpy.AlterField_management(in_fc, "POINT_Y", "Y_ETRS_UTM31", "Northing [ETRS 1989 UTM31]")


# Create an update cursor to access the rows of the feature class
with arcpy.da.UpdateCursor(in_fc, ["X_ETRS_UTM31", "Y_ETRS_UTM31"]) as cursor:
    # Loop through each row
    for row in cursor:
        # Round the coordinates to 1 decimal place using numpy.round
        row[0] = numpy.round(row[0], 0)
        row[1] = numpy.round(row[1], 0)
        # Update the row with the rounded values
        cursor.updateRow(row)


# ETRS1989 - 4258
# Set the output coordinate system
out_sr = arcpy.SpatialReference(4258) 
geom_props = "POINT_X_Y_Z_M"
arcpy.AddGeometryAttributes_management(in_fc, geom_props , "", "", out_sr)


# Rename the fields POINT_X and POINT_Y to X_COORD and Y_COORD
arcpy.AlterField_management(in_fc, "POINT_X", "X_ETRS", "X [ETRS 1989]")
arcpy.AlterField_management(in_fc, "POINT_Y", "Y_ETRS", "Y [ETRS 1989]")

# Create a search cursor to fetch data from the feature class
fields = [field.name for field in arcpy.ListFields(in_fc)]
print('Fields:',fields)



Fields: ['OBJECTID', 'Shape', 'X_ETRS_UTM31', 'Y_ETRS_UTM31', 'POINT_Z', 'X_ETRS', 'Y_ETRS']


#### C) Export to SHP and Excel

In [8]:

# Extract directory and file name
file_path = os.path.dirname(in_fc)
file_name = in_fc.split(os.path.sep)[-1]
excel_name = file_name +".xlsx"
shp_name = file_name +".shp"


ouput_excel = os.path.join(directory_excel, excel_name)
output_shapefile = os.path.join(directory_excel, shp_name)

# print("Full Path:", in_fc)
# print("Directory:", file_path)
print("Excel Path:", ouput_excel)
print("Shp Path:", output_shapefile)


Excel Path: \\WM20ocqu46ph01\WF_Projects\NL_HKW\4_OUTPUT\_Innovation\HKW_VII_Subsea_Battery_pt_UTM31N_v2\HKW_VII_Subsea_Battery_pt_UTM31N_v2.xlsx
Shp Path: \\WM20ocqu46ph01\WF_Projects\NL_HKW\4_OUTPUT\_Innovation\HKW_VII_Subsea_Battery_pt_UTM31N_v2\HKW_VII_Subsea_Battery_pt_UTM31N_v2.shp


In [9]:
if not os.path.exists(directory_excel):
    os.makedirs(directory_excel)
    print(f"Folder '{directory_excel}' created successfully.")
else:
    print(f"Folder '{directory_excel}' already exists.")


# Export attribute table to Excel
arcpy.TableToExcel_conversion(in_fc, ouput_excel)
# Export feature class to shapefile
arcpy.CopyFeatures_management(in_fc, output_shapefile)

print("Attribute table:", output_shapefile)
print("Shapefile:", ouput_excel )



Folder '\\WM20ocqu46ph01\WF_Projects\NL_HKW\4_OUTPUT\_Innovation\HKW_VII_Subsea_Battery_pt_UTM31N_v2' already exists.
Attribute table: \\WM20ocqu46ph01\WF_Projects\NL_HKW\4_OUTPUT\_Innovation\HKW_VII_Subsea_Battery_pt_UTM31N_v2\HKW_VII_Subsea_Battery_pt_UTM31N_v2.shp
Shapefile: \\WM20ocqu46ph01\WF_Projects\NL_HKW\4_OUTPUT\_Innovation\HKW_VII_Subsea_Battery_pt_UTM31N_v2\HKW_VII_Subsea_Battery_pt_UTM31N_v2.xlsx
